Tính các trọng số cầu phương $c_i$ của từng đa thức

In [15]:
import numpy as np
import sympy as sp
from sympy import integrate

## ________Cài đặt các thư viện cần thiết________

# pip install numpy
# pip install sympy

x = sp.symbols('x')

def tich_trong(w, f, g, a, b):
    return integrate(w(x)*f*g, (x, a, b))

def initial(w, a_coef, b_coef, polynomial):
    a_coef.append(None)
    b_coef.extend([None, None]) # b_0 = 0
    polynomial.append(1) # P_0(x) = 1
    a_coef.append(tich_trong(w, x * polynomial[0], polynomial[0], a, b) / tich_trong(w, polynomial[0], polynomial[0], a, b)) # a_1

def compute_a_coefficient(w, n, coef, polynomial):
    tmp = tich_trong(w, x * polynomial[n - 1], polynomial[n - 1], a, b) / tich_trong(w, polynomial[n - 1], polynomial[n - 1], a, b)
    coef.append(tmp)

def compute_b_coefficient(w, n, coef, polynomial):
    if n < 2:
        return
    tmp = tich_trong(w, x * polynomial[n - 1], polynomial[n - 2], a, b) / tich_trong(w, polynomial[n - 2], polynomial[n - 2], a, b)
    coef.append(tmp)

def P_n(x, n, polynomial, a_coef, b_coef): # m = n - 1
    return (x - a_coef[n]) * polynomial[n - 1] - b_coef[n] * polynomial[n - 2]

def kiem_tra_truc_giao(w, Phi, n):
    mat = np.zeros((n + 1, n + 1))
    for i in range(0, n + 1):
        for j in range(0, n + 1):
            tmp = tich_trong(w, Phi[i], Phi[j], a, b)
            mat[i][j] = tmp
            print(tmp, end = "\t")
        print()

def generate_polynomial(w, polynomial, a_coef, b_coef):
    # P1
    initial(w, a_coef, b_coef, polynomial)
    polynomial.append(x - a_coef[1])
    for i in range(2, n + 1):
        compute_a_coefficient(w, i, a_coef, polynomial)
        compute_b_coefficient(w, i, b_coef, polynomial)
        polynomial.append(P_n(x, i, polynomial, a_coef, b_coef).simplify())


def w1(x):
    return 1
def w2(x):
    return 1 / sp.sqrt(1 - x**2)
def w3(x):
    return sp.exp(-x)
def w4(x):
    return sp.exp(-x**2)

n = 5
weight_function = [w1, w2, w3, w4]
grid = [(-1, 1), (-1, 1), (0, sp.oo), (-sp.oo, sp.oo)]
name = ["Legrendree", "Chebyshev", "Laguerre", "Hermite"]
short_name = ["P", "T", "L", "H"]

# convert sympy function to latex
def convert_latex(function):
    return sp.latex(function)

for i in range(len(name)):
    a, b = grid[i][0], grid[i][1]
    a_coef, b_coef, polynomial = [], [], []
    generate_polynomial(weight_function[i], polynomial, a_coef, b_coef)
    with open("tmp.md", "w") as f:
        print(name[i], file = f)
        for j in range(0, n):
            print(f"$$n = {j}$$", file = f)
            x_i = sp.solve(polynomial[j + 1])
            C = []
            for k in range(len(x_i)):
                tmp = 1
                for l in range(len(x_i)):
                    if l != k:
                        tmp *= (x - x_i[l]) / (x_i[k] - x_i[l])
                print("$$ x_ " +  str(k) + "=" + sp.latex(x_i[k]) + ";C_" + str(k) + "=" + "\\int_{" + str(a) + "}^{" + str(b) +  "}" + sp.latex(tmp) + "dx = "  +  sp.latex(integrate(tmp, (x, a, b))) + "$$", file = f)
                tmp = integrate(tmp, (x, a, b))
                C.append(tmp.simplify())
            print(file = f)
    break
    